In [2]:
from pathlib import Path
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

DATASET = Path("datasets/lotto/data_processed.csv")

# The Answer to the Ultimate Question of Life, the Universe, and Everything.
np.random.seed(42)

In [3]:
# Read into data frame
dataset = pd.read_csv(DATASET, header=0, sep=',', quotechar='"', parse_dates=['DrawDate'], dtype={'PrizeType': str})

In [4]:
df = dataset.copy()

In [5]:
for ind, column in enumerate(df.columns):
    print('"{}"'.format(column), end=', ')
    if ind % 8 == 0:
        print()

"DrawNo", 
"DrawDate", "PrizeType", "LuckyNo", "DaysSinceLastExactMatch", "DaysUntilNextExactMatch", "DaysSinceLastAnyMatch", "DaysUntilNextAnyMatch", "1st_digit", 
"2nd_digit", "3rd_digit", "4th_digit", "DrawYear", "DrawMonth", "DrawWeek", "DrawDay", "DrawDayofweek", 
"DrawDayofyear", "CumProbaExactMatch", "CumProbaAnyMatch", "0_1", "0_2", "0_3", "0_4", "0_5", 
"0_6", "0_7", "0_8", "0_9", "1_2", "1_3", "1_4", "1_5", 
"1_6", "1_7", "1_8", "1_9", "2_3", "2_4", "2_5", "2_6", 
"2_7", "2_8", "2_9", "3_4", "3_5", "3_6", "3_7", "3_8", 
"3_9", "4_5", "4_6", "4_7", "4_8", "4_9", "5_6", "5_7", 
"5_8", "5_9", "6_7", "6_8", "6_9", "7_8", "7_9", "8_9", 
"0_0", "1_1", "2_2", "3_3", "4_4", "5_5", "6_6", "7_7", 
"8_8", "9_9", "pos_1_0_freq", "pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", "pos_1_4_freq", "pos_1_5_freq", 
"pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", "pos_2_1_freq", "pos_2_2_freq", "pos_2_3_freq", 
"pos_2_4_freq", "pos_2_5_freq", "pos_2_6_freq", "pos_

In [6]:
columns_to_drop = [ "DaysSinceLastExactMatch", "DaysUntilNextAnyMatch",
                    "1st_digit", "2nd_digit", "3rd_digit", "4th_digit",
                    "0_1", "0_2", "0_3", 
                    "0_4", "0_5", "0_6", "0_7", "0_8", "0_9", "1_2", "1_3", 
                    "1_4", "1_5", "1_6", "1_7", "1_8", "1_9", "2_3", "2_4", 
                    "2_5", "2_6", "2_7", "2_8", "2_9", "3_4", "3_5", "3_6", 
                    "3_7", "3_8", "3_9", "4_5", "4_6", "4_7", "4_8", "4_9", 
                    "5_6", "5_7", "5_8", "5_9", "6_7", "6_8", "6_9", "7_8", 
                    "7_9", "8_9", "0_0", "1_1", "2_2", "3_3", "4_4", "5_5", 
                    "6_6", "7_7", "8_8", "9_9",
                    "pos_1_0_freq", "pos_1_1_freq", "pos_1_2_freq", "pos_1_3_freq", 
                    "pos_1_4_freq", "pos_1_5_freq", "pos_1_6_freq", "pos_1_7_freq", "pos_1_8_freq", "pos_1_9_freq", "pos_2_0_freq", "pos_2_1_freq", 
                    "pos_2_2_freq", "pos_2_3_freq", "pos_2_4_freq", "pos_2_5_freq", "pos_2_6_freq", "pos_2_7_freq", "pos_2_8_freq", "pos_2_9_freq", 
                    "pos_3_0_freq", "pos_3_1_freq", "pos_3_2_freq", "pos_3_3_freq", "pos_3_4_freq", "pos_3_5_freq", "pos_3_6_freq", "pos_3_7_freq", 
                    "pos_3_8_freq", "pos_3_9_freq", "pos_4_0_freq", "pos_4_1_freq", "pos_4_2_freq", "pos_4_3_freq", "pos_4_4_freq", "pos_4_5_freq", 
                    "pos_4_6_freq", "pos_4_7_freq", "pos_4_8_freq", "pos_4_9_freq"
                  ]
df.drop(columns_to_drop,inplace=True, axis=1, errors='ignore')
df.columns

Index(['DrawNo', 'DrawDate', 'PrizeType', 'LuckyNo', 'DaysUntilNextExactMatch',
       'DaysSinceLastAnyMatch', 'DrawYear', 'DrawMonth', 'DrawWeek', 'DrawDay',
       'DrawDayofweek', 'DrawDayofyear', 'CumProbaExactMatch',
       'CumProbaAnyMatch'],
      dtype='object')

In [7]:
df.describe()

,DrawNo,LuckyNo,DaysUntilNextExactMatch,DaysSinceLastAnyMatch,DrawYear,DrawMonth,DrawWeek,DrawDay,DrawDayofweek,DrawDayofyear,CumProbaExactMatch,CumProbaAnyMatch
count,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000,104673.000000
mean,268299.492419,4974.001777,764.409447,66.179053,2005.754779,6.501648,26.452428,15.628214,4.054054,182.380356,0.062382,0.930387
std,131388.886981,2885.324727,832.538166,93.325465,7.809743,3.462904,15.123744,8.942459,1.843042,105.868193,0.038001,0.675621
min,40792.000000,0.000000,0.000000,0.000000,1992.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.010000,0.010000
25%,154499.000000,2463.000000,157.000000,15.000000,1999.000000,3.000000,13.000000,8.000000,2.000000,90.000000,0.030000,0.380000
50%,268306.000000,4952.000000,504.000000,39.000000,2006.000000,7.000000,26.000000,16.000000,5.000000,182.000000,0.060000,0.800000
75%,382113.000000,7463.000000,1095.000000,83.000000,2013.000000,10.000000,40.000000,24.000000,6.000000,274.000000,0.090000,1.310000
max,495819.000000,9999.000000,8087.000000,4113.000000,2019.000000,12.000000,53.000000,31.000000,6.000000,366.000000,0.270000,2.930000
